## Tiny-Agent
- step1:构造大模型
- step2:构造工具
- step3:构造Agent
- step4:运行Agent

In [ ]:
!pip install dashscope

### step1:构造大模型

In [8]:
%%writefile .env
OPENAI_API_KEY='sk-7INmRu6iHEope6cJCa462b608aFb4eE281Df64F4E32e95F9'
OPENAI_BASE_URL='https://threefive.gpt7.link/v1'  # 有些国内的代理商需要加v1，有些不需要

ZHIPUAI_API_KEY='68604cc3e382eba3aba03daca6f4f717.fteCwuYKtXWpexN6'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'

Writing .env


In [38]:
from typing import Dict, List, Optional, Tuple, Union
os.environ['CURL_CA_BUNDLE'] = ''
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

class BaseModel:
    def __init__(self, path: str = '') -> None:
        self.path = path

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        pass

    def load_model(self):
        pass
    
class DashscopeChat(BaseModel):
    def __init__(self, path: str = '', model: str = "qwen-turbo") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[Dict], content: str) -> str:
        import dashscope
        dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")
        #history.append({'role': 'user', 'content': prompt})
        response = dashscope.Generation.call(
            model=self.model,
            messages=history,
            result_format='message',
            max_tokens=150,
            temperature=0.1
        )
        return response.output.choices[0].message.content,history
    

### Step 2: 构造工具

In [32]:
import os, json
import requests

"""
工具函数

- 首先要在 tools 中添加工具的描述信息
- 然后在 tools 中添加工具的具体实现

- https://serper.dev/dashboard
"""

class Tools:
    def __init__(self) -> None:
        self.toolConfig = self._tools()
    
    def _tools(self):
        tools = [
            {
                'name_for_human': '谷歌搜索',
                'name_for_model': 'google_search',
                'description_for_model': '谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。',
                'parameters': [
                    {
                        'name': 'search_query',
                        'description': '搜索关键词或短语',
                        'required': True,
                        'schema': {'type': 'string'},
                    }
                ],
            }
        ]
        return tools

    def google_search(self, search_query: str):
        url = "https://google.serper.dev/search"

        payload = json.dumps({"q": search_query})
        headers = {
            'X-API-KEY': 'd8ba7d116c5dd2154ed7d5ba08caea5b0d4c0197',
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload).json()

        return response['organic'][0]['snippet']


### Step 3: 构造Agent

In [95]:
from typing import Dict, List, Optional, Tuple, Union
import json5

TOOL_DESC = """{name_for_model}: Call this tool to interact with the {name_for_human} API. What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters} Format the arguments as a JSON object."""
REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tool_descs}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""


class Agent:
    def __init__(self, path: str = '') -> None:
        self.path = path
        self.tool = Tools()
        self.system_prompt = self.build_system_input()
        self.model = DashscopeChat(path)
        self.history=[{'role':'system','content':self.system_prompt}]

    def build_system_input(self):
        tool_descs, tool_names = [], []
        for tool in self.tool.toolConfig:
            tool_descs.append(TOOL_DESC.format(**tool))
            tool_names.append(tool['name_for_model'])
        tool_descs = '\n\n'.join(tool_descs)
        tool_names = ','.join(tool_names)
        sys_prompt = REACT_PROMPT.format(tool_descs=tool_descs, tool_names=tool_names)
        return sys_prompt
    
    def parse_latest_plugin_call(self, text):
        plugin_name, plugin_args = '', ''
        i = text.rfind('\nAction:')
        j = text.rfind('\nAction Input:')
        k = text.rfind('\nObservation:')
        if 0 <= i < j:  # If the text has `Action` and `Action input`,
            if k < j:  # but does not contain `Observation`,
                text = text.rstrip() + '\nObservation:'  # Add it back.
            k = text.rfind('\nObservation:')
            plugin_name = text[i + len('\nAction:') : j].strip()
            plugin_args = text[j + len('\nAction Input:') : k].strip()
            text = text[:k]
        return plugin_name, plugin_args, text
    
    def call_plugin(self, plugin_name, plugin_args):
        plugin_args = json5.loads(plugin_args)
        if plugin_name == 'google_search':
            return '\nObservation:' + self.tool.google_search(**plugin_args)

    def text_completion(self, text,history=None):
        if history is not None:
            self.history=history
        text = "\nQuestion:" + text
        self.history.append({'role':'user','content':text})
        response, self.history = self.model.chat(text, self.history,self.system_prompt )
        plugin_name, plugin_args, response = self.parse_latest_plugin_call(response)
        # print(plugin_name)
        if plugin_name:
            response += self.call_plugin(plugin_name, plugin_args)
        print(response)
        self.history.append({'role':'user','content':response})
        response1, self.history = self.model.chat(response, self.history, self.system_prompt)
        self.history.append({'role':'assistant','content':response1})
        return response1,self.history

if __name__ == '__main__':
    agent = Agent('')
    prompt = agent.build_system_input()
    print(prompt)

Answer the following questions as best you can. You have access to the following tools:

google_search: Call this tool to interact with the 谷歌搜索 API. What is the 谷歌搜索 API useful for? 谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。 Parameters: [{'name': 'search_query', 'description': '搜索关键词或短语', 'required': True, 'schema': {'type': 'string'}}] Format the arguments as a JSON object.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [google_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!



In [90]:
agent = Agent('')
response, _ = agent.text_completion(text='你好')
print(response)

Thought: 我会回答。
Final Answer: 你好！有什么我可以帮助你的吗？
```


In [96]:
agent = Agent('')
response, _ = agent.text_completion(text='周杰伦哪一年出生的？', history=None)
print(response)

Thought: 我需要查询周杰伦的出生年份。
Action: google_search
Action Input: {"search_query": "周杰伦 出生年份"}
Observation:周杰伦（1979年1月18日—），台湾男歌手、演员、词曲作家及制作人。自2000年推出首张音乐专辑《Jay》出道，他的音乐风行于大中华地区及全球各地的华人社群，并对华语 ...
Thought: 从搜索结果中，我找到了周杰伦的出生年份是1979年。
Final Answer: 周杰伦出生于1979年。


In [81]:
agent = Agent('')
response, _ = agent.text_completion(text='周杰伦是谁？', history=None)
print(response)

Thought: 我需要查询周杰伦的相关信息。
Action: google_search
Action Input: {"search_query": "周杰伦"}
Observation:周杰倫（1979年1月18日—），臺灣男歌手、演員、詞曲作家及製作人。自2000年推出首張音樂專輯《Jay》出道，他的音樂風行於大中華地區及全球各地的華人社群，並對華語 ...
Thought: 我现在知道了周杰伦的相关信息。
Final Answer: 周杰伦是一位来自台湾的歌手、演员、词曲作家和制作人。他于2000年发行了首张音乐专辑《Jay》正式出道，并在华语地区以及全球华人社群中流行起来，对华语音乐界产生了重要影响。


In [83]:
response, _ = agent.text_completion(text='书生浦语是什么？', history=None)
print(response)

Thought: 我需要查找“书生浦语”这个词语的相关信息。
Action: google_search
Action Input: {"search_query": "书生浦语"}
Observation:致力于大模型研究与开发工具链的开源组织。 为所有AI 开发者提供高效、易用的开源平台，让最前沿的大模型与算法技术触手可 ...
Thought: 搜索结果没有提供关于“书生浦语”的具体信息，这可能是一个特定的术语或者地方用语，需要进一步的上下文来理解其含义。
Final Answer: “书生浦语”可能是一个特定的术语或者地方用语，但根据当前搜索结果无法确定其确切含义。建议提供更多的上下文信息或者检查输入是否正确。
